In [1]:
import tensorflow as tf
from keras import datasets,Sequential
from keras.layers import Conv2D,Dense,MaxPooling2D,Flatten
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import densenet
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Activation, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [3]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [4]:
#------------------------------------------------------------------------------
# VGG16 ON CIFAR_10
#------------------------------------------------------------------------------
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16
import tensorflow.keras as k
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from keras.utils.np_utils import to_categorical
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score

In [5]:
model_vanilla = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,(3,3), activation='relu', input_shape=(32,32,3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='same'),
    tf.keras.layers.BatchNormalization(axis = 3),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2),padding='same'),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='same'),
    tf.keras.layers.BatchNormalization(axis = 3),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2),padding='same'),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Conv2D(128,(3,3),activation='relu',padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128,(3,3),activation='relu',padding='same'),
    tf.keras.layers.BatchNormalization(axis = 3),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2),padding='same'),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(100, activation='softmax')
])

In [6]:

model_vanilla.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 30, 30, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 30, 30, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                        

In [7]:
(X_train, y_train), (X_test, y_test) = k.datasets.cifar100.load_data()

print("******************")
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

169001437/169001437 [==============================] - 6s 0us/step
******************
(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [8]:
# Convert class vectors to binary class matrices using one hot encoding
y_train_ohe = to_categorical(y_train, num_classes = 100)
y_test_ohe = to_categorical(y_test, num_classes = 100)

In [9]:
# Data normalization
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train  /= 255
X_test /= 255

print("******************")
print(X_train.shape)
print(y_train_ohe.shape)
print(X_test.shape)
print(y_test_ohe.shape)

******************
(50000, 32, 32, 3)
(50000, 100)
(10000, 32, 32, 3)
(10000, 100)


In [10]:
X_val = X_train[40000:]
y_val = y_train_ohe[40000:]
print(X_val.shape)
print(y_val.shape)

(10000, 32, 32, 3)
(10000, 100)


In [11]:
X_train = X_train[:40000]
y_train_ohe = y_train_ohe[:40000]
print(X_train.shape)
print(y_train_ohe.shape)

(40000, 32, 32, 3)
(40000, 100)


In [12]:
model_vanilla.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [13]:
aug = ImageDataGenerator(
    rotation_range=20, 
    zoom_range=0.15, 
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    shear_range=0.15,
    horizontal_flip=True, 
    fill_mode="nearest")

# train the model
"""
history = model_vanilla.fit_generator(
    aug.flow(X_train,y_train_ohe, batch_size=128),
    validation_data=(X_val,y_val),
    steps_per_epoch=len(X_train) // 128,
    epochs=50,
    callbacks=[reduce_lr,mc])
"""


'\nhistory = model_vanilla.fit_generator(\n    aug.flow(X_train,y_train_ohe, batch_size=128),\n    validation_data=(X_val,y_val),\n    steps_per_epoch=len(X_train) // 128,\n    epochs=50,\n    callbacks=[reduce_lr,mc])\n'

In [14]:
import os

In [15]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [16]:
models_dir = "saved_models"
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

checkpointer = ModelCheckpoint(filepath='saved_models/model_vanilla.hdf5', 
                               monitor='val_accuracy', mode='max',
                               verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=2, min_lr=0.001)
callbacks=[early_stopping, reduce_lr, checkpointer]

In [17]:
# Training model
import time
training_start = time.time()

history1 = model_vanilla.fit(aug.flow(X_train,y_train_ohe, batch_size=128),
          epochs=50,validation_data=(X_val,y_val),callbacks=callbacks)
training_stop = time.time()
training_time = training_stop - training_start
print(f"Training time: {training_time}")
#history1 = model_vanilla.fit(train_ds, epochs = 20, validation_data = val_ds, callbacks=callbacks)

Epoch 1/50
313/313 [==============================] - ETA: 0s - loss: 4.5133 - accuracy: 0.0434
Epoch 1: val_accuracy improved from -inf to 0.01640, saving model to saved_models/model_vanilla.hdf5
313/313 [==============================] - 38s 70ms/step - loss: 4.5133 - accuracy: 0.0434 - val_loss: 5.0740 - val_accuracy: 0.0164 - lr: 0.0010
Epoch 2/50
313/313 [==============================] - ETA: 0s - loss: 3.9532 - accuracy: 0.0863
Epoch 2: val_accuracy improved from 0.01640 to 0.11770, saving model to saved_models/model_vanilla.hdf5
313/313 [==============================] - 21s 68ms/step - loss: 3.9532 - accuracy: 0.0863 - val_loss: 3.7759 - val_accuracy: 0.1177 - lr: 0.0010
Epoch 3/50
313/313 [==============================] - ETA: 0s - loss: 3.7340 - accuracy: 0.1209
Epoch 3: val_accuracy improved from 0.11770 to 0.17640, saving model to saved_models/model_vanilla.hdf5
313/313 [==============================] - 21s 67ms/step - loss: 3.7340 - accuracy: 0.1209 - val_loss: 3.4100 -

In [18]:
input_shape=(32,32,3)

In [19]:
vgg16 = VGG16(weights="imagenet", include_top=False, input_shape=input_shape)
vgg16.trainable = False
inputs = tf.keras.Input(input_shape)
x = vgg16(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(100, activation='softmax')(x)
model_vgg16 = tf.keras.Model(inputs, x)

58889256/58889256 [==============================] - 2s 0us/step


In [20]:
model_vgg16.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_vgg16.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense_3 (Dense)             (None, 1024)              525312    
                                                                 
 dense_4 (Dense)             (None, 512)               524800    
                                                                 
 dense_5 (Dense)             (None, 100)               51300     
                                                             

In [21]:
checkpointer = ModelCheckpoint(filepath='saved_models/model_vgg16.hdf5', 
                               monitor='val_accuracy', mode='max',
                               verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=2, min_lr=0.001)
callbacks=[early_stopping, reduce_lr, checkpointer]

In [22]:
history2 = model_vgg16.fit(aug.flow(X_train,y_train_ohe, batch_size=128),
          epochs=50,validation_data=(X_val,y_val),callbacks=callbacks)

Epoch 1/50
313/313 [==============================] - ETA: 0s - loss: 3.5686 - accuracy: 0.1629
Epoch 1: val_accuracy improved from -inf to 0.24510, saving model to saved_models/model_vgg16.hdf5
313/313 [==============================] - 24s 71ms/step - loss: 3.5686 - accuracy: 0.1629 - val_loss: 3.0501 - val_accuracy: 0.2451 - lr: 0.0010
Epoch 2/50
313/313 [==============================] - ETA: 0s - loss: 3.0935 - accuracy: 0.2411
Epoch 2: val_accuracy improved from 0.24510 to 0.28480, saving model to saved_models/model_vgg16.hdf5
313/313 [==============================] - 21s 68ms/step - loss: 3.0935 - accuracy: 0.2411 - val_loss: 2.8744 - val_accuracy: 0.2848 - lr: 0.0010
Epoch 3/50
313/313 [==============================] - ETA: 0s - loss: 2.9683 - accuracy: 0.2625
Epoch 3: val_accuracy improved from 0.28480 to 0.29650, saving model to saved_models/model_vgg16.hdf5
313/313 [==============================] - 21s 68ms/step - loss: 2.9683 - accuracy: 0.2625 - val_loss: 2.8078 - val_a

In [23]:
model_vgg16.load_weights('saved_models/model_vgg16.hdf5')

In [24]:
val_loss, val_accuracy = model_vgg16.evaluate(X_val, y_val)
print('Validation loss: {}\nValidation accuracy: {}'.format(val_loss, val_accuracy))

313/313 [==============================] - 1s 3ms/step - loss: 2.5880 - accuracy: 0.3483
Validation loss: 2.5880045890808105
Validation accuracy: 0.3483000099658966


In [25]:
test_loss, test_accuracy = model_vgg16.evaluate(X_test,y_test_ohe,)
print('Testing loss: {}\nTesting accuracy: {}'.format(test_loss, test_accuracy))

313/313 [==============================] - 1s 3ms/step - loss: 2.5832 - accuracy: 0.3505
Testing loss: 2.58323335647583
Testing accuracy: 0.3504999876022339
